# Global Solution 2 - RPA

Integrantes:<br>
- Henrique Marra Barbosa - RM97672
- Arthur Hieda Cunha - RM551882
- Lucas Bueno Taets Gustavo - RM552162

### Imports

In [ ]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### Configurações

In [74]:
# Configurações de APIs
ENERGY_API_URL = "https://api.eia.gov/v2/electricity/retail-sales/data/"
EIA_API_KEY = "gEdTQHlS1PksCEtK9LC6mz1BwO7luKW5YdJ1Yppg"
WEATHER_API_URL = "https://api.weatherapi.com/v1/current.json"
WEATHER_API_KEY = "90cc6a6e12b941108a5233745241811"

# Banco de Dados SQLite
DB_NAME = "energia_sustentavel.db"

### Coleta de Dados

In [ ]:
def fetch_energy_data():
    params = {
        "api_key": EIA_API_KEY,
        "frequency": "monthly",
        "data[0]": "price",
        "facets[stateid][]": "CA",
        "start": "2022-01",
        "end": "2022-12",
        "sort[0][column]": "period",
        "sort[0][direction]": "asc"
    }
    response = requests.get(ENERGY_API_URL, params=params)
    if response.status_code == 200:
        return response.json()['response']['data']
    else:
        raise Exception(f"Erro ao acessar a API da EIA: {response.status_code}")

def fetch_weather_data():
    weather_data = []
    for month in range(1, 13):
        date = f"2022-{month:02d}-01"
        params = {"key": WEATHER_API_KEY, "q": "São Paulo", "dt": date}
        response = requests.get(WEATHER_API_URL, params=params)
        if response.status_code == 200:
            weather_data.append(response.json())
        else:
            print(f"Erro ao acessar a API de clima para {date}: {response.status_code}")
    return weather_data

def scrape_energy_news():
    url = "https://www.energy.gov/eere/articles"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    news_items = soup.find_all('h3', class_='field-content')
    return [item.text.strip() for item in news_items[:5]]  # Retorna os 5 primeiros itens de notícias

### Tratamento de Dados

In [76]:
def process_data(energy_data, weather_data, news_data):
    # Dados de energia
    energy_df = pd.DataFrame(energy_data)
    energy_df['period'] = pd.to_datetime(energy_df['period'])
    energy_df = energy_df[['period', 'price']].sort_values('period')
    
    # Dados de clima
    weather_df = pd.DataFrame(weather_data)
    weather_df['date'] = pd.to_datetime(weather_df['date'])
    weather_df = weather_df.sort_values('date')
    
    # Notícias de energia
    news_df = pd.DataFrame(news_data, columns=['news'])
    news_df['date'] = pd.date_range(start='2022-01-01', periods=len(news_df), freq='M')
    
    # Padronização e união dos dados
    merged_df = pd.merge_asof(energy_df, weather_df, left_on='period', right_on='date')
    merged_df = pd.merge_asof(merged_df, news_df, left_on='period', right_on='date')
    
    return merged_df

### Armazenamento e Modelagem

In [77]:
def save_to_storage(dataframe):
    # Exportar para CSV
    dataframe.to_csv("energia_sustentavel.csv", index=False)
    
    # Salvar no SQLite
    conn = sqlite3.connect(DB_NAME)
    dataframe.to_sql("consumo_energia", conn, if_exists="replace", index=False)
    conn.close()

def train_model(dataframe):
    X = dataframe[['temp_c', 'humidity']]
    y = dataframe['price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    score = model.score(X_test, y_test)
    print(f"Precisão do modelo: {score:.2f}")
    
    return model

def plot_predictions(model, dataframe):
    plt.figure(figsize=(10, 6))
    plt.scatter(dataframe['temp_c'], dataframe['price'], color='blue', label='Dados reais')
    plt.plot(dataframe['temp_c'], model.predict(dataframe[['temp_c', 'humidity']]), color='red', label='Previsão')
    plt.xlabel('Temperatura (°C)')
    plt.ylabel('Preço da Energia')
    plt.title('Previsão de Preço da Energia baseada na Temperatura')
    plt.legend()
    plt.savefig('previsao_energia.png')
    plt.close()

### Execução Principal

In [78]:
if __name__ == "__main__":
    try:
        # Coletar dados das fontes
        energy_data = fetch_energy_data()
        weather_data = fetch_weather_data()
        news_data = scrape_energy_news()

        # Processar os dados
        processed_data = process_data(energy_data, weather_data, news_data)

        # Armazenar os dados
        save_to_storage(processed_data)

        # Treinar modelo e fazer previsões
        model = train_model(processed_data)
        plot_predictions(model, processed_data)

        print("Processo concluído com sucesso!")
    except Exception as e:
        print(f"Erro: {e}")

Dados climáticos não encontrados para 2022-01-01
Dados climáticos não encontrados para 2022-01-31
Dados climáticos não encontrados para 2022-03-02
Dados climáticos não encontrados para 2022-04-01
Dados climáticos não encontrados para 2022-05-01
Dados climáticos não encontrados para 2022-05-31
Dados climáticos não encontrados para 2022-06-30
Dados climáticos não encontrados para 2022-07-30
Dados climáticos não encontrados para 2022-08-29
Dados climáticos não encontrados para 2022-09-28
Dados climáticos não encontrados para 2022-10-28
Dados climáticos não encontrados para 2022-11-27
Erro: 'date'


In [94]:
import requests
import sqlite3
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time

# Configurações de APIs
ENERGY_API_URL = "https://api.eia.gov/v2/electricity/retail-sales/data/"
EIA_API_KEY = "gEdTQHlS1PksCEtK9LC6mz1BwO7luKW5YdJ1Yppg"
OPENWEATHER_API_KEY = "af32a9a5b87e5bfb1e5d9c63ac81300c"
LATITUDE = 36.7783  # Latitude da Califórnia
LONGITUDE = -119.4179  # Longitude da Califórnia

# Banco de Dados SQLite
DB_NAME = "energia_sustentavel.db"

# Função para converter data para timestamp UNIX
def convert_date_to_timestamp(date_str):
    date_time_obj = time.strptime(date_str, "%Y-%m-%d")
    return int(time.mktime(date_time_obj))

# Função para buscar dados de energia
def fetch_energy_data():
    params = {
        "api_key": EIA_API_KEY,
        "frequency": "monthly",
        "data[0]": "price",
        "facets[stateid][]": "CA",
        "start": "2022-01",
        "end": "2022-12",
        "sort[0][column]": "period",
        "sort[0][direction]": "asc"
    }
    response = requests.get(ENERGY_API_URL, params=params)
    if response.status_code == 200:
        return response.json()['response']['data']
    else:
        raise Exception(f"Erro ao acessar a API da EIA: {response.status_code}")

# Função para buscar dados do clima usando OpenWeatherAPI
def fetch_weather_data():
    weather_data = []
    for month in range(1, 13):
        date = f"2022-{month:02d}-01"  # Formato da data
        params = {
            "lat": LATITUDE,
            "lon": LONGITUDE,
            "appid": OPENWEATHER_API_KEY,
            "units": "metric"  # Para obter a temperatura em Celsius
        }
        # Usando o endpoint de clima atual, já que OpenWeather não tem um endpoint simples para dados históricos mensais
        response = requests.get("https://api.openweathermap.org/data/2.5/weather", params=params)

        print(f"Requisição para {date}: {response.url}")  # Imprime a URL da requisição
        print(f"Status Code: {response.status_code}")   # Imprime o código de status da resposta
        
        if response.status_code == 200:
            try:
                weather_info = response.json()
                print(f"Resposta da API para {date}: {weather_info}")  # Imprime a resposta completa
                
                # Extraindo dados de clima
                weather_data.append({
                    "date": date,
                    "temp_c": weather_info['main']['temp'],  # Temperatura média do dia em Celsius
                    "humidity": weather_info['main']['humidity']  # Umidade média
                })
            except KeyError as e:
                print(f"Erro ao acessar dados para {date}: {e}")
        else:
            print(f"Erro ao acessar a API de clima para {date}: {response.status_code}")
    
    return weather_data

# Função para processar dados
def process_data(energy_data, weather_data):
    # Dados de energia
    energy_df = pd.DataFrame(energy_data)
    energy_df['period'] = pd.to_datetime(energy_df['period'])
    energy_df = energy_df[['period', 'price']].sort_values('period')
    
    # Dados de clima
    weather_df = pd.DataFrame(weather_data)
    weather_df['date'] = pd.to_datetime(weather_df['date'])
    weather_df = weather_df.sort_values('date')
    
    # Padronização e união dos dados
    # Usando a função merge_asof para garantir o alinhamento temporal
    merged_df = pd.merge_asof(energy_df, weather_df, left_on='period', right_on='date')
    
    # Verifique se há alguma falta de dados após a mesclagem
    merged_df = merged_df.dropna()  # Remove linhas com dados faltantes

    return merged_df

# Função para salvar dados
def save_to_storage(dataframe):
    # Exportar para CSV
    dataframe.to_csv("energia_sustentavel.csv", index=False)
    
    # Salvar no SQLite
    conn = sqlite3.connect(DB_NAME)
    dataframe.to_sql("consumo_energia", conn, if_exists="replace", index=False)
    conn.close()

# Função para treinar modelo
def train_model(dataframe):
    # Selecione as variáveis independentes (temperatura e umidade)
    X = dataframe[['temp_c', 'humidity']]
    
    # Selecione a variável dependente (preço da energia)
    y = dataframe['price']
    
    # Divisão dos dados em treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Inicialize o modelo de regressão linear
    model = LinearRegression()
    
    # Treinamento do modelo
    model.fit(X_train, y_train)
    
    # Avaliação do modelo
    score = model.score(X_test, y_test)
    print(f"Precisão do modelo: {score:.2f}")
    
    return model

# Função para plotar previsões
def plot_predictions(model, dataframe):
    plt.figure(figsize=(10, 6))
    plt.scatter(dataframe['temp_c'], dataframe['price'], color='blue', label='Dados reais')
    plt.plot(dataframe['temp_c'], model.predict(dataframe[['temp_c', 'humidity']]), color='red', label='Previsão')
    plt.xlabel('Temperatura (°C)')
    plt.ylabel('Preço da Energia')
    plt.title('Previsão de Preço da Energia baseada na Temperatura')
    plt.legend()
    plt.savefig('previsao_energia.png')
    plt.close()

if __name__ == "__main__":
    try:
        # Coletar dados das fontes
        energy_data = fetch_energy_data()
        weather_data = fetch_weather_data()

        # Processar os dados
        processed_data = process_data(energy_data, weather_data)

        # Armazenar os dados
        save_to_storage(processed_data)

        # Treinar modelo e fazer previsões
        model = train_model(processed_data)
        plot_predictions(model, processed_data)

        print("Processo concluído com sucesso!")
    except Exception as e:
        print(f"Erro: {e}")


Requisição para 2022-01-01: https://api.openweathermap.org/data/2.5/weather?lat=36.7783&lon=-119.4179&appid=af32a9a5b87e5bfb1e5d9c63ac81300c&units=metric
Status Code: 200
Resposta da API para 2022-01-01: {'coord': {'lon': -119.4179, 'lat': 36.7783}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 12.45, 'feels_like': 11.32, 'temp_min': 11.71, 'temp_max': 13.71, 'pressure': 1018, 'humidity': 60, 'sea_level': 1018, 'grnd_level': 992}, 'visibility': 10000, 'wind': {'speed': 2.06, 'deg': 300}, 'clouds': {'all': 100}, 'dt': 1731979098, 'sys': {'type': 1, 'id': 5145, 'country': 'US', 'sunrise': 1731940779, 'sunset': 1731977198}, 'timezone': -28800, 'id': 5325369, 'name': 'Avocado', 'cod': 200}
Requisição para 2022-02-01: https://api.openweathermap.org/data/2.5/weather?lat=36.7783&lon=-119.4179&appid=af32a9a5b87e5bfb1e5d9c63ac81300c&units=metric
Status Code: 200
Resposta da API para 2022-02-01: {'coord': {'lon':